# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv ("weather_info.csv")
weather = weather.drop("Unnamed: 0", axis = 1)
weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
0,iqaluit,35.60,63.75,-68.51,74,18.34,75,CA
1,vostok,45.34,46.49,135.88,78,1.72,99,RU
2,barra bonita,75.20,-22.49,-48.56,41,3.36,0,BR
3,hermanus,52.00,-34.42,19.23,97,5.17,100,ZA
4,isangel,75.20,-19.55,169.27,83,5.82,75,VU
...,...,...,...,...,...,...,...,...
562,tianpeng,79.00,30.99,103.94,1,6.29,43,CN
563,necochea,48.00,-38.55,-58.74,84,3.00,3,AR
564,jalalpur pirwala,81.00,29.51,71.22,33,15.08,0,PK
565,nizhnyaya tavda,48.54,57.67,66.17,91,9.35,100,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

from ipywidgets.embed import embed_minimal_html
locations = weather[['Lat', 'Lng']]
weights=weather['Humidity']


# map types ['ROADMAP', 'HYBRID', 'TERRAIN', 'SATELLITE']
fig = gmaps.figure(map_type='TERRAIN', center=(0,0), zoom_level =3) 

heat_layer = gmaps.heatmap_layer(locations, weights=weights,max_intensity=0,point_radius = 30)

# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Make sure max temperature is not higher than 75 or lower than 60
* Ensure cloudiness is 0
* Wind is less than 10 mph

In [6]:
vacation_weather = weather.loc[(weather["Max Temp"]>60)&(weather["Max Temp"]<=75)]
vacation_weather = vacation_weather.loc[vacation_weather["Wind Speed"]<=10]
vacation_weather = vacation_weather.loc[vacation_weather["Cloudiness"]==0]
vacation_weather = vacation_weather.fillna("NM")
vacation_weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
23,chuy,61.68,-33.70,-53.46,82,7.23,0,UY
30,caxias do sul,60.01,-29.17,-51.18,41,1.86,0,BR
51,libertador general san martin,65.75,-23.81,-64.79,57,3.51,0,AR
139,sao joao da barra,72.54,-21.64,-41.05,85,7.00,0,BR
146,henties bay,70.72,-22.12,14.28,39,6.51,0,NM
164,cidreira,64.29,-30.18,-50.21,71,8.01,0,BR
173,coxim,69.46,-18.51,-54.76,61,1.48,0,BR
185,kabalo,68.11,-6.05,26.92,68,4.65,0,CD
210,tiznit,67.53,29.58,-9.50,57,1.57,0,MA
274,kasulu,62.56,-4.58,30.10,94,1.59,0,TZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = vacation_weather.copy()
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df.rename(columns = {"Cities":"City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
23,chuy,61.68,-33.70,-53.46,82,7.23,0,UY,
30,caxias do sul,60.01,-29.17,-51.18,41,1.86,0,BR,
51,libertador general san martin,65.75,-23.81,-64.79,57,3.51,0,AR,
139,sao joao da barra,72.54,-21.64,-41.05,85,7.00,0,BR,
146,henties bay,70.72,-22.12,14.28,39,6.51,0,NM,
164,cidreira,64.29,-30.18,-50.21,71,8.01,0,BR,
173,coxim,69.46,-18.51,-54.76,61,1.48,0,BR,
185,kabalo,68.11,-6.05,26.92,68,4.65,0,CD,
210,tiznit,67.53,29.58,-9.50,57,1.57,0,MA,
274,kasulu,62.56,-4.58,30.10,94,1.59,0,TZ,


In [15]:

target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    long = row["Lng"]
    params = {
        "location": f'{lat},{long}',
        "radius":5000,
        "keyword":"hotel",
        "key": g_key
    }


    response = requests.get(target_url, params).json()

    results = response['results']
    
    try:
        print(f"First hotel near {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print(response)
        hotel_df.loc[index, 'Hotel Name'] = "No Hotel Found within 5000 m"
        
    print("------------")

First hotel near chuy is Bertelli Chuí Hotel.
------------
First hotel near caxias do sul is Hotel Intercity Caxias do Sul.
------------
First hotel near libertador general san martin is Jardín Colonial.
------------
First hotel near sao joao da barra is Pousada Kactus.
------------
First hotel near henties bay is Desert Rose.
------------
First hotel near cidreira is Pond Country Club.
------------
First hotel near coxim is Hotel Beira Rio.
------------
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
------------
Missing field/result... skipping.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
------------
First hotel near kasulu is High Way Hotel.
------------
First hotel near manono is Hotel hewa bora.
------------
First hotel near chokwe is Hotel Sorriso de Caridade.
------------
First hotel near nacala is Nacala Plaza Business Design Hotel.
------------
First hotel near bela vista do paraiso is Hotel Pasettu's

In [9]:
hotel_df
hotel_df = hotel_df.rename(columns = {"Cities": "City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
23,chuy,61.68,-33.70,-53.46,82,7.23,0,UY,Bertelli Chuí Hotel
30,caxias do sul,60.01,-29.17,-51.18,41,1.86,0,BR,Hotel Intercity Caxias do Sul
51,libertador general san martin,65.75,-23.81,-64.79,57,3.51,0,AR,Jardín Colonial
139,sao joao da barra,72.54,-21.64,-41.05,85,7.00,0,BR,Pousada Kactus
146,henties bay,70.72,-22.12,14.28,39,6.51,0,NM,Desert Rose
164,cidreira,64.29,-30.18,-50.21,71,8.01,0,BR,Pond Country Club
173,coxim,69.46,-18.51,-54.76,61,1.48,0,BR,Hotel Beira Rio
185,kabalo,68.11,-6.05,26.92,68,4.65,0,CD,No Hotel Found within 5000 m
210,tiznit,67.53,29.58,-9.50,57,1.57,0,MA,No Hotel Found within 5000 m
274,kasulu,62.56,-4.58,30.10,94,1.59,0,TZ,High Way Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

  
fig.add_layer(markers)
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))